# Opérateurs de requête - évaluation: `$expr`, `$regex`, `$mod`, `$text`, `$where`

Ce notebook présente les **opérateurs d'évaluation** de MongoDB à travers des exemples simples et accessibles.  
L'objectif est de montrer comment filtrer des documents à l'aide de conditions d'évaluation sur les différents champs de données dans une base MongoDB.

Nous allons :
- Créer une connexion à MongoDB local
- Insérer des données fictives
- Illustrer les opérateurs d'évaluation un par un

## 1. Connexion à la base de données

Avant toute opération sur une base de données, il est nécessaire de s'y connecter:

In [87]:
from pymongo import MongoClient
import pandas as pd

# Définition du port à utiliser [à changer en fonction la configuration de votre installation locale]
MONGODB_LOCAL_PATH = "mongodb://localhost:27018/"

# Définition de la base de données à utiliser
MONGODB_NAME = "evaluation_operator_db"

# Connexion
client = MongoClient(MONGODB_LOCAL_PATH)
db = client[MONGODB_NAME]

## 2. Création d'une collection pour un environnement de test

Les commandes suivantes permettent de créer une collection servant d'environnement de test pour les différentes opérations de comparaison de ce notebook

In [88]:
try:
    # Création de la collection
    db_player = db.create_collection("Player")
except:
    # Suppression puis création de la collection
    db.Player.drop()
    db_player = db.create_collection("Player")

On va remplir cette collection avec un petit nombre de documents

In [ ]:
documents = [
    {
        "name": "Bazooka Bob",
        "attack": 32,
        "defence": 18,
        "credo": "À l'attaque !",
        "objet_fétiche": "bazooka",
        "équipements": ["gilet par balle", "couteau", "casque"]
    },
    {
        "name": "Karate Kid",
        "attack": 26,
        "defence": 24,
        "credo": "Si tu m'attaques, je riposte.",
        "objet_fétiche": "bandana",
        "équipements": []
    },
    {
        "name": "Richard d'acier",
        "attack": 10,
        "defence": 35,
        "credo": "Défense d'acier",
        "objet_fétiche": "armure",
        "équipements": ["pistolet", "couteau", "casque"]
    },
    {
        "name": "Ninja Nina",
        "attack": 15,
        "defence": 27,
        "credo": "Niak ! Ninja !",
        "objet_fétiche": "couteau",
        "équipements": ["shuriken", "tenue de camouflage"]
    },
    {
        "name": "Major Maurice",
        "attack": 19,
        "defence": 19,
        "credo": "La meilleure défense, c'est l'attaque.",
        "objet_fétiche": "pistolet",
        "équipements": ["fusil", "casque", "gilet par balle"]
    },
]
result_many = db_player.insert_many(documents)
print("IDs inséré pour les nouveaux documents:\n", result_many.inserted_ids)

IDs inséré pour les nouveaux documents:
 [ObjectId('6888a36163a3ef32fc02c5aa'), ObjectId('6888a36163a3ef32fc02c5ab'), ObjectId('6888a36163a3ef32fc02c5ac'), ObjectId('6888a36163a3ef32fc02c5ad'), ObjectId('6888a36163a3ef32fc02c5ae')]


## 3. L'opérateur `$expr`

Ce premier opérateur permet de filtrer les documents en précisant une **expression de recherche pouvant combiner plusieurs champs**:

`{ $expr: { <expression> } }`

---

Par exemple:
>  "Je recherche les joueurs qui ont plus d'attaque que de défense"

In [ ]:
pd.DataFrame(db_player.find({"$expr": {"$gt": ["$attack", "$defence"]}}).to_list())

,_id,name,attack,defense,credo,objet_fétiche,équipements
0,6888a36163a3ef32fc02c5aa,Bazooka Bob,32,18,À l'attaque !,bazooka,"[gilet par balle, couteau, casque]"
1,6888a36163a3ef32fc02c5ab,Karate Kid,26,24,"Si tu m'attaques, je riposte.",bandana,[]


## 4. L'opérateur `$regex`

Cet opérateur permet de **filtrer les documents en précisant une expression régulière** que la valeur d'un champ de données doit respecter:

`{ <field>: { $regex: "pattern", $options: '<options>' } }`

Il existe plusieurs options à découvrir dans la documentation MongoDB. Pour l'exemple ci-dessous, l'option **i** permet de rendre la recherche insensible à la casse.

---

Par exemple:
>  "Je recherche les joueurs dont le credo contient la mention de 'attaque'"

In [91]:
pd.DataFrame(db_player.find({"credo": {"$regex": "attaque", "$options": "i"}}).to_list())

,_id,name,attack,defense,credo,objet_fétiche,équipements
0,6888a36163a3ef32fc02c5aa,Bazooka Bob,32,18,À l'attaque !,bazooka,"[gilet par balle, couteau, casque]"
1,6888a36163a3ef32fc02c5ab,Karate Kid,26,24,"Si tu m'attaques, je riposte.",bandana,[]
2,6888a36163a3ef32fc02c5ae,Major Maurice,19,19,"La meilleure défense, c'est l'attaque.",pistolet,"[fusil, casque, gilet par balle]"


## 5. L'opérateur `$mod`

Cet opérateur permet de filtrer les documents suivant le résultat de l'**application de l'opération mathématique modulo** sur un champ de données. L'opérateur s'invoque comme suit:

`{ field: { $mod: [ divisor, remainder ] } }`

---

Par exemple:
> "Je recherche les joueurs dont le montant de l'attaque ou de la défense est pair"

In [ ]:
pd.DataFrame(db_player.find(
    {"$or": [
        {"attack": {"$mod": [2,0]}}, 
        {"defence": {"$mod": [2,0]}},
    ]}
).to_list())

,_id,name,attack,defense,credo,objet_fétiche,équipements
0,6888a36163a3ef32fc02c5aa,Bazooka Bob,32,18,À l'attaque !,bazooka,"[gilet par balle, couteau, casque]"
1,6888a36163a3ef32fc02c5ab,Karate Kid,26,24,"Si tu m'attaques, je riposte.",bandana,[]
2,6888a36163a3ef32fc02c5ac,Richard d'acier,10,35,Défense d'acier,armure,"[pistolet, couteau, casque]"


## 6. L'opérateur `$text`

Cet opérateur permet de **filtrer les documents en faisant une recherche textuelle**. L'opérateur s'invoque comme suit:

```
{
  $text: {
    $search: <string>,
    $language: <string>,
    $caseSensitive: <boolean>,
    $diacriticSensitive: <boolean>
  }
}
```
---

**Attention: cet opérateur n'est applicable que sur du texte indexé**

Les commandes suivantes permettent d'obtenir un index textuel à partir d'un champ de données de notre collection.

In [93]:
text_index = db_player.create_index({"credo": "text"})

Ainsi l'opérateur `$text` est disponible pour ce champ.

Par exemple:
> "Je recherche les joueurs dont le credo contient la mention de 'defense'"

In [ ]:
pd.DataFrame(db_player.find({"$text": {"$search": "defense"}}).to_list())

,_id,name,attack,defense,credo,objet_fétiche,équipements
0,6888a36163a3ef32fc02c5ac,Richard d'acier,10,35,Défense d'acier,armure,"[pistolet, couteau, casque]"
1,6888a36163a3ef32fc02c5ae,Major Maurice,19,19,"La meilleure défense, c'est l'attaque.",pistolet,"[fusil, casque, gilet par balle]"


Il n'est pas possible d'avoir plusieurs indexes textuels en même temps, du coup pour en créer un nouveau, il faut supprimer le premier.

In [95]:
db_player.drop_index(text_index)

On peut aussi créer un index textuel en combinant plusieurs champs de données.

In [96]:
text_index = db_player.create_index(
    {
        "objet_fétiche": "text",
        "équipements": "text",
    }
)

Ainsi l'opérateur `$text` est disponible pour une recherche dans ces deux champs en même temps.

Par exemple:
> "Je recherche les joueurs qui possèdent un couteau"

In [97]:
pd.DataFrame(db_player.find({"$text": {"$search": "couteau"}}).to_list())

,_id,name,attack,defense,credo,objet_fétiche,équipements
0,6888a36163a3ef32fc02c5ad,Ninja Nina,15,27,Niak ! Ninja !,couteau,"[shuriken, tenue de camouflage]"
1,6888a36163a3ef32fc02c5ac,Richard d'acier,10,35,Défense d'acier,armure,"[pistolet, couteau, casque]"
2,6888a36163a3ef32fc02c5aa,Bazooka Bob,32,18,À l'attaque !,bazooka,"[gilet par balle, couteau, casque]"


## L'opérateur `$where`

L'opérateur `$where` permet d'apporter de la flexibilité dans les requêtes via l'**utilisation de fonctions JavaScript**. Il s'invoque via le schéma suivant:

`{ $where: <string|JavaScript Code> }`

*Conseil: en général, si la requête peut être faite via les opérateurs MongoDB, il est préférable de choisir ces derniers pour des questions de performance dans l'éxecution de la requête.*

---

Par exemple:
> "Je veux récupérer les joueurs dont le MD5 hash de leur nom commence par une lettre plus grande que 'd'"

In [112]:
pd.DataFrame(db_player.find({
    "$where": """function() {
        return (hex_md5(this.name) > 'd')
    }"""
}).to_list())

,_id,name,attack,defense,credo,objet_fétiche,équipements
0,6888a36163a3ef32fc02c5aa,Bazooka Bob,32,18,À l'attaque !,bazooka,"[gilet par balle, couteau, casque]"
1,6888a36163a3ef32fc02c5ae,Major Maurice,19,19,"La meilleure défense, c'est l'attaque.",pistolet,"[fusil, casque, gilet par balle]"


---

## Conclusion

Dans ce notebook, nous avons exploré certains **opérateurs d'évaluation** de MongoDB :  
- `$expr` : permet d'exprimer une expression d'évaluation combinant plusieurs champs dans une requête
- `$regex` : permet l'emploi d'expression régulière dans l'évaluation d'un champ
- `$mod` : permet l'emploi de l'opérateur mathématique modulo dans l'évaluation d'un champ numérique
- `$text` : permet une recherche textuelle à partir d'un index textuel préalablement établi
- `$where` : permet de faire des requêtes plus avancées grâce à l'emploi de fonction JavaScripy

Chaque opérateur a été présenté avec un exemple simple et compréhensible, en se basant sur une collection de joueurs fictifs.

Ce travail permet de mieux comprendre comment MongoDB peut évaluer les champs de données contenus dans un document.